# Implementing the ICAR Model in Stan


## Notebook Setup

Import all libraries, load the NYC study data.

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import libpysal as sa
import matplotlib
import splot as splt
from splot.libpysal import plot_spatial_weights 
import plotnine as p9
import arviz as az
%matplotlib inline

from cmdstanpy import CmdStanModel, cmdstan_path, cmdstan_version

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=12),
        axis_title_y=p9.element_text(size=12),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

map_theme =  p9.theme(figure_size=(7,6),
                 axis_text_x=p9.element_blank(),
                 axis_ticks_x=p9.element_blank(),
                 axis_text_y=p9.element_blank(),
                 axis_ticks_y=p9.element_blank())

In [ ]:
nyc_geodata = gpd.read_file(os.path.join('data', 'nyc_study.geojson'))
nyc_geodata.columns

## Disconnected Components (and islands)

New York city consists of several islands; only the Bronx is part of the mainland; Brooklyn and Queens are part of Long Island, plus smaller islands City Island, Roosevelt Island, and the Rockaways.
**Python**

In [ ]:
w_nyc_nbs = sa.weights.Rook.from_dataframe(nyc_geodata, geom_col='geometry')
print(f'number of components: {w_nyc_nbs.n_components}')
print(f'islands? {w_nyc_nbs.islands}')
print(f'max number of neighbors per node: {w_nyc_nbs.max_neighbors}')
print(f'mean number of neighbors per node: {w_nyc_nbs.mean_neighbors}')

*This is a problem for the ICAR model, which operates on a fully connected graph (single component)*

* For the NYC analysis paper, we hand edited the map of NYC (in R) to create a fully connected network graph.

* For this notebook we restrict out attention to Brooklyn, the largest borough in NYC, which is a single network component.

In [ ]:
brooklyn_geodata = nyc_geodata[nyc_geodata['BoroName']=='Brooklyn'].reset_index(drop=True)
brooklyn_nbs = sa.weights.Rook.from_dataframe(brooklyn_geodata, geom_col='geometry')
plot_spatial_weights(brooklyn_nbs, brooklyn_geodata) 

In [ ]:
print(f'number of components: {brooklyn_nbs.n_components}')
print(f'islands? {brooklyn_nbs.islands}')
print(f'max number of neighbors per node: {brooklyn_nbs.max_neighbors}')
print(f'mean number of neighbors per node: {brooklyn_nbs.mean_neighbors}')

## Adding an ICAR Component to the Base Model

The ICAR model uses the spatial structure of the data, abstracted into a neighborhood network.

### Alternative encodings of neighborhood network

- $N \times N$ Adjacency matrix
  + Entries $(i,\ j)$ and $(j,\ i)$ are 1 when regions $n_i$ and $n_j$ are neighbors, 0 otherwise

- Undirected graph: regions are vertices, pairs of neighbors are edges
  + Encoded as *edges* - 2 column matrix, each row is a pair of neighbors $({n_i}, {n_j})$

The Stan ICAR model uses the latter.

### Stan ICAR Model Implementation

$$ p(\phi) \propto \exp \left\{ {- \frac{1}{2}} \sum_{i \sim j}{({\phi}_i - {\phi}_j)}^2 \right\} $$

Use Stan's vectorized operations to compute log probability density:
\fontsize{9pt}{9.4}\selectfont
```
   target += -0.5 * dot_self(phi[node1] - phi[node2]);
```
\normalsize
\vspace{0.2in}

Encode neighbor information as a graph.

* Nodes are indexed from 1:N.
* Edges indices are stored in a 2 x N array
  + each column is an edge
  + row 1: index of first node in edge pair, $n_i$
  + row 2: index of second node in edge pair, $n_j$

\fontsize{9pt}{9.4}\selectfont
```
  int<lower = 0> N;  // number of areal regions
  // spatial structure
  int<lower = 0> N_edges;  // number of neighbor pairs
  array[2, N_edges] int<lower = 1, upper = N> neighbors;  // node[1, j] adjacent to node[2, j]
```
\normalsize

## ICAR Model:  `poisson_icar.stan`

This file is in directory `stan/poisson_icar.stan`.

In [ ]:
poisson_icar_model_file = os.path.join('stan', 'poisson_icar.stan')

with open(poisson_icar_model_file, 'r') as file:
    contents = file.read()
    print(contents)

## Data Prep

#### Get edgeset

- Compute this automatically from `nyc_geodata` spatial geometry component
  + Python package `libpysal`
  + R package `spdep`

In [ ]:
brooklyn_nbs_adj =  brooklyn_nbs.to_adjlist(remove_symmetric=True)
print(type(brooklyn_nbs_adj))
brooklyn_nbs_adj.head(10)

In [ ]:
# create np.ndarray from columns in adjlist, increment indices by 1
j1 = brooklyn_nbs_adj['focal'] + 1
j2 = brooklyn_nbs_adj['neighbor'] + 1
edge_pairs = np.vstack([j1, j2])
edge_pairs

#### Assemble the input data

In [ ]:
design_vars = np.array(['pct_pubtransit','med_hh_inc', 'traffic', 'frag_index'])

design_mat = brooklyn_geodata[design_vars].to_numpy()
design_mat[:, 1] = np.log(design_mat[:, 1])
design_mat[:, 2] = np.log(design_mat[:, 2])

pd.DataFrame(data=design_mat).describe()

In [ ]:
pois_icar_data = {"N":brooklyn_geodata.shape[0],
             "y":brooklyn_geodata['count'].astype('int'),
             "E":brooklyn_geodata['kid_pop'].astype('int'),
             "K":4,
             "xs":design_mat,
             "N_edges": edge_pairs.shape[1],
             "neighbors": edge_pairs
}

## Fitting the ICAR Model on the Brooklyn data

#### Model is compiled (as needed) on instantiation

In [ ]:
pois_icar_mod = CmdStanModel(stan_file=poisson_icar_model_file)

#### Run the NUTS-HMC sampler, summarize results

In [ ]:
pois_icar_fit = pois_icar_mod.sample(data=pois_icar_data)
pois_icar_fit.summary().round(2).loc[
  ['beta_intercept', 'beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]', 'sigma']]

## Model Comparison: ICAR vs. ordinary random effects

In [ ]:
pois_re_mod = CmdStanModel(stan_file=os.path.join(
  'stan', 'poisson_re.stan'))
pois_re_fit = pois_re_mod.sample(data=pois_icar_data)
pois_re_fit.summary().round(2).loc[
  ['beta_intercept', 'beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]', 'sigma']]

Which model provides a better fit (on the Brooklyn subset of the data)?

### Visual comparison

#### ICAR model

In [ ]:
idata_pois_icar = az.from_cmdstanpy(
    pois_icar_fit,
    posterior_predictive="y_rep",
    dims={"betas": ["covariates"]},
    coords={"covariates": design_vars},
    observed_data={"y": pois_icar_data['y']}
)
idata_pois_icar

az.plot_ppc(idata_pois_icar, data_pairs={"y":"y_rep"})

#### RE model

In [ ]:
idata_pois_re = az.from_cmdstanpy(
    pois_re_fit,
    posterior_predictive="y_rep",
    dims={"betas": ["covariates"]},
    coords={"covariates": design_vars},
    observed_data={"y": pois_icar_data['y']}
)
az.plot_ppc(idata_pois_re, data_pairs={"y":"y_rep"})

### Leave-one-out cross-validation (LOO)

The Poisson-ICAR model falls into the category of "non-factorized models",
for which simple leave-one-out cross-validation fails to hold,
due to the fact that observations depend on other observations from 
different spatial units in addition to the dependence on the model parameters
Doing cross-validation would require recoding the model, and then re-fitting is $N$ times, once per omitted observation.

See paper: [Efficient leave-one-out cross-validation for Bayesian non-factorized normal and Student-t models](https://arxiv.org/abs/1810.10559),
which provides a thorough discussion of this problem and ways to evaluate non-factorized models,
and the LOO package vignette:
[Leave-one-out cross-validation for non-factorized models](https://mc-stan.org/loo/articles/loo2-non-factorized.html).

In these notebooks we will rely on visualizations and summary statistics.
